# Juego de la Vida (Game of Life)

El **Juego de la Vida**, también conocido como **Vida**, es un autómata celular diseñador por el matemático británico John Horton Conway en 1970. Es un juego sin jugadores, esto significa que su evolución está determinada por su estado inicial, sin requerir más entradas. Uno interactúa con el Juego de la Vida al crear un estado inicial (o configuración inicial) y se observa como evoluciona. Es Turing completo (puede simular cualquier máquina de Turing) y puede simular un constructor universal o cualquier otra máquina de Turing.

<center>
<img src="https://1.bp.blogspot.com/-pdQ2ckeQzCw/WwGFbYTKRoI/AAAAAAAAAY8/kb-gcutnkKc2TnQainAlYK5FlWmv87XywCPcBGAYYCw/s1600/ezgif.com-video-to-gif.gif" width=250 height=250 />
</center>

## Reglas

El universo (o entorno) del Juego de la Vida es infinito, cuadrícula ortogonal bidimensional de celdas cuadradas, cada una de ellas se encuentra con uno de dos posibles estados, vivo o muerto (poblado o despoblado, respectivamente). Cada celda interactúa con ocho vecinos, que son las celdas adyacentes horizontales, verticales y diagonales. En cada paso de tiempo, las siguientes transiciones ocurren:

1. Cualquier celda viva con menos de dos vecinos vivos muere, debido a la subpoblación.
2. Cualquier celda viva con dos o tres vecinos vivos sobrevive para la siguiente generación.
3. Cualquier celda con más de tres vecinos vivos muere, debido a la sobrepoblación.
4. Cualquier celda muerta con exactamente tres vecinos vivos se convierte en una celda viva, debido a la reproducción.

Estas reglas, que comparan el comportamiento del autómata a la vida real, pueden ser condensadas en lo siguiente:

1. Cualquier celda viva con dos o más vecinos vivos sobrevive.
2. Cualquier celda muerta con tres vecinos vivos se convierte en una celda viva.
3. Cualquier otra celda viva muere en la siguiente generación. De manera similar, cualquier otra celda muerta se queda muerta.

El patrón inicial constituye la semilla del sistema. La primera generación es creada al aplicar las reglas anteriores de manera simultanea a cualquier celda en la semilla, viva o muerta; nacimientos y muertes ocurren simultáneamente, y el momento discreto en el cual esto pasa es a veces llamado turno. Cada generación es una función pura de la anterior. Las reglas se aplican de forma repetida para crear nuevas generaciones.

## Imports

Antes de empezar a crear el modelo del juego de la vida con multiagentes es necesario tener instalado los siguientes paquetes:
- `python`: asegúrense de usar la versión 3+.
- `mesa`: el framework de Python para el modelado de agentes.
- `numpy`: es una biblioteca de Python para el manejo de matrices, arreglos, manipulación matemática, lógica y mucho más.
- `matplotlib`: es una biblioteca para crear visualizaciones estáticas, animadas e interactivas en Python.

Para poder modelar el juego de la vida usando el framework de `mesa` es necesario importar dos clases: una para el modelo general, y otro para los agentes. 

In [ ]:
# 'Model' sirve para definir los atributos a nivel del modelo, maneja los agentes
# 'Agent' es la unidad atómica y puede ser contenido en múltiples instancias en los modelos
from mesa import Agent, Model 

# 'MultiGrid' sirve para que los agentes sean capaces de sobreponerse en una sola celda
from mesa.space import MultiGrid

# 'SimultaneousActivation' habilita la opción de activar todos los agentes de manera simultanea.
from mesa.time import SimultaneousActivation

# 'DataCollector' permite obtener el grid completo a cada paso (o generación), útil para visualizar
from mesa.datacollection import DataCollector

# 'matplotlib' lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2 ** 128

# Definimos los siguientes paquetes para manejar valores númericos: 'numpy' & 'pandas'
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

## Crear el modelo

Antes que nada el presente modelo se encuentra basado en el [tutorial introductorio](https://mesa.readthedocs.io/en/latest/tutorials/intro_tutorial.html). Se modificó para que funcionara con el presente problema pero en esencia es lo mismo.

In [ ]:
class AgentBarredora(Agent):
    """
    Representa a los agentes de limpieza. 
    """
    def __init__(self, u_id, ubi, model):
        """
        Crea un agente al que se le asigna un identificador
        formado por una tupla (x,y).
        """
        super().__init__(u_id, model)
        self.pos = ubi
    
    def step(self):
        """
        Este método es el que busca celdas sucias en la proximidad para después
        moverse hacia ellas y limpiarlas.
        """
        neighbours = self.model.grid.get_neighbors(
            self.pos,
            moore=True,
            include_center=True)

        # Detectar celda sucia y limpiarla -------------------------------------
        for obj in neighbours:
            if isinstance(obj, AgentCelda) and obj.pos == self.pos:
                obj.state = 0

        # Moverse a otra celda -------------------------------------------------
        n = True
        p = 0
        while (n == True) and (p < len(neighbours)):
            neighbor = neighbours[p]
            # Revisa si hay celdas sucias a su alrededor
            if isinstance(neighbor, AgentCelda) and neighbor.state == 1:
                n = False
                next_pos = neighbor.pos
            p += 1

        if n == True:
            # Seleccionar nueva posicion aleatoria
            next_pos = (np.random.choice(neighbours)).pos
        
        self.model.grid.move_agent(self, next_pos)
        
                
class AgentCelda(Agent):
    """
    Representa a un agente o una celda con estado sucio (1) o limpio (0).
    """
    def __init__(self, unique_id, model, state):
        """
        Crea un agente con estado inicial aleatorio de 0 ó 1, también se le asigna un identificador
        formado por una tupla (x,y).
        """
        super().__init__(unique_id, model)
        self.pos = unique_id
        self.state = state
        self.next_state = None
    
    def step(self):
        """
        La celda no necesita hacer nada más que existir.
        """

            
class ModelHabitacion(Model):
    """
    Define el modelo de la simulación de máquinas barredoras.
    """
    def __init__(self, width, height):
        self.num_agents = width * height
        self.grid = MultiGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)
        
        for (content, x, y) in self.grid.coord_iter():
            a = AgentCelda((x, y), self, np.random.randint(0,2))
            self.grid.place_agent(a, (x, y))
            self.schedule.add(a)

        id = 100
        for x in range(0, width, 2):
            ubi = (x, 0)
            b = AgentBarredora(id, ubi, self)
            self.grid.place_agent(b, (x, 0))
            self.schedule.add(b)
            id += 10
        
        # Aquí definimos el colector de datos para obtener el grid completo.
        self.datacollector = DataCollector(
            model_reporters={"Grid": self.get_grid}
        )
    
    def step(self):
        """
        En cada paso el colector toma la información que se definió y almacena el grid para luego
        graficarlo.
        """
        self.datacollector.collect(self)
        self.schedule.step()

    def get_grid(self):
        """
        Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
        :param model: El modelo del cual obtener el grid.
        :return: Matriz con la información del grid del agente.
        """

        # Generamos la grid para contener los valores
        grid = np.zeros((self.grid.width, self.grid.height))

        # Asignamos una celda a cada uno de los elementos de la grilla
        for cell in self.grid.coord_iter():
            cell_content, x, y = cell
            for obj in cell_content:
                if isinstance(obj, AgentBarredora):
                    grid[x][y] = 2
                elif isinstance(obj, AgentCelda):
                    grid[x][y] = obj.state
          
        return grid



# Ejecución del modelo
A continuación corremos el modelo

In [ ]:
# Definimos el tamaño del Grid
GRID_SIZE = 7

# Definimos el número de generaciones a correr
NUM_GENERATIONS = 20

# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()
model = ModelHabitacion(GRID_SIZE, GRID_SIZE)
for i in range(NUM_GENERATIONS):
    model.step()

# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

Tiempo de ejecución: 0:00:00.013173


Obtenemos la información que almacenó el colector, este nos entregará un DataFrame de pandas que contiene toda la información.

In [ ]:
all_grid = model.datacollector.get_model_vars_dataframe()

Graficamos la información usando `matplotlib`

In [ ]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])

cmap = matplotlib.cm.get_cmap('viridis', 3)
cmap = cmap(np.linspace(0, 1, 3))
cmap[0] = np.array([255/256, 255/256, 255/256, 1])   # celdas limpias
cmap[1] = np.array([0/256, 0/256, 0/256, 1])    # celdas sucias
cmap[2] = np.array([30/256, 144/256, 255/256, 1])  # barredora

new_cmap = matplotlib.colors.ListedColormap(cmap)

patch = plt.imshow(all_grid.iloc[0][0], cmap = new_cmap)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=NUM_GENERATIONS)

In [ ]:
anim

In [ ]:
!pip install mesa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 11.0 MB/s 
     |████████████████████████████████| 64 kB 1.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
